## Setup

In [ ]:
from specific import *

### Get shifted data

In [ ]:
(
    endog_data,
    exog_data,
    master_mask,
    filled_datasets,
    masked_datasets,
    land_mask,
) = get_offset_data()

### Retrieve previous results from the 'model' notebook

In [ ]:
results, rf = cross_val_cache.load()

### Predict BA

In [ ]:
n_threads = get_ncpus()
rf.n_jobs = n_threads
with parallel_backend("threading", n_jobs=n_threads):
    predicted_ba = get_masked_array(rf.predict(exog_data), master_mask)
target_ba = get_masked_array(endog_data.values, master_mask)

In [ ]:
ba_predicted = predicted_ba
ba_data = target_ba

model_name = "RF1"

# Plotting params.
figsize = (5.1, 2.8)
mpl.rcParams["figure.figsize"] = figsize
coast_linewidth = 0.3
date_str = "2010-01 to 2015-01"

In [ ]:
boundaries = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]


# Plotting predicted.
fig = cube_plotting(
    ba_predicted,
    cmap="brewer_RdYlBu_11_r",
    title=f"Predicted BA\n{date_str}",
    boundaries=boundaries,
    colorbar_kwargs={"label": "Burned Area Fraction", "format": "%0.0e"},
    coastline_kwargs={"linewidth": coast_linewidth},
)
figure_saver.save_figure(fig, f"predicted_ba_{model_name}", sub_directory="predictions")

# Plotting observed.
fig = cube_plotting(
    ba_data,
    cmap="brewer_RdYlBu_11_r",
    title=f"Observed BA\n{date_str}",
    boundaries=boundaries,
    colorbar_kwargs={"label": "Burned Area Fraction", "format": "%0.0e"},
    coastline_kwargs={"linewidth": coast_linewidth},
)
figure_saver.save_figure(fig, f"observed_ba_{model_name}", sub_directory="predictions")

In [ ]:
frac_diffs = (ba_data - ba_predicted) / ba_data

In [ ]:
# Plotting differences.
diff_boundaries = [-1e1, -1e0, 0, 1e-1]

fig = cube_plotting(
    frac_diffs,
    cmap="brewer_RdYlBu_11",
    title=f"BA Discrepancy (Obs. - Pred.) / Obs. \n{date_str}",
    cmap_midpoint=0,
    boundaries=diff_boundaries,
    colorbar_kwargs={"label": "(Obs. - Pred.) / Obs.", "format": "%0.0e"},
    coastline_kwargs={"linewidth": coast_linewidth},
)
figure_saver.save_figure(
    fig, f"difference_ba_{model_name}", sub_directory="predictions"
)